In [1]:
from matplotlib.pylab import*

from scipy.interpolate import griddata
from matplotlib import cm
import pandas as pd
from time import perf_counter
from sympy import*
import warnings
warnings.filterwarnings('ignore')

from Inputs import*
    
order = array([2,3])        # polynomial order
Nv = array([2,4])
kstages = 3
cfl = 0.25
dt = 1e-2
Tfinal = 0.5
iplot = False            # plot the solution
iconverg = False

time_method = "BDF3"      # IRK, BDF2 or BDF3
integration_type = 1      # % = 1 is inexact and = 2 is exact
icase = 4                 # select icase: 1,2,3,4

alpha = 1              
beta = 0#-1/2                  # Dirichlet: alpha = 0, beta = 1
                               # Neumann: alpha = 1, beta = 0
                               # Robin: alpha = 1, beta != 0

x_boundary = [4,4]    # Bottom and Top (x = -1 and x = +1)
y_boundary = [6,6]    # Left and Right (y = -1 and x = +1)


#output_file = 'output.dat'
#file = open(output_file,"wb")
#file.write(array([len(Nv)]))
#Visualisation(order,Nv,time_method,kstages,integration_type,icase,Tfinal,alpha,beta,\
#              x_boundary,y_boundary,iplot,iconverg)

#file.close()

Visualisation2(order,Nv,time_method,kstages,integration_type,icase,Tfinal,alpha,beta,x_boundary,y_boundary)

Problem: Diffusion
Domain: [-1, 1]
Diffusivity: 0.01
Boundary conditions: Neumann

N = 2, nel = 4, Np = 25
	dt = 3.3333e-02
	Number of time steps = 15
N = 2, nel = 16, Np = 81
	dt = 1.0000e-02
	Number of time steps = 50
N = 3, nel = 4, Np = 49
	dt = 1.6667e-02
	Number of time steps = 30
N = 3, nel = 16, Np = 169
	dt = 4.7619e-03
	Number of time steps = 105

End simulation


In [2]:
import pandas 

idx = pandas.IndexSlice

# Integration  : 
# time steps   : Number of time steps to time tfinal
# dt           : Time step size
# CFL          : CFL number
# Errors       : (1-norm, 2-norm and inf-norm) errors

ex_list = ['Diffusion']

cols = ['time steps', 'cfl','2-norm']
elements = Nv
iterables = [ex_list, ['CG'],['exact','inexact'], order, elements]

index = pandas.MultiIndex.from_product(iterables,names=['Eq.','method','integration','order','elements'])
df = pandas.DataFrame(index=index,columns=cols).sort_index()

df.loc[idx['Diffusion','CG',['inexact']]]

time steps  cfl 2-norm
Eq.       method integration order elements                       
Diffusion CG     inexact     2     2               NaN  NaN    NaN
                                   4               NaN  NaN    NaN
                             3     2               NaN  NaN    NaN
                                   4               NaN  NaN    NaN

In [3]:
output_file = 'output.dat'

# Open file as a binary file
fout = open(output_file,"rb")  
nlen = fromfile(fout,dtype=int,count=1)

# Create a data type for storing results;  use 'd' for double precision entries
dt_data = dtype([('N',int),('M',int),('cfl','d'),('2-norm','d')])

# Create a nested data type
dt_info = dtype([('order',int),('integType',int)])
dt_block = dtype([('info',dt_info),('data',(dt_data,nlen))])

# Read in four sets of data (orders 1,2 and integType 1,2)
out_data = fromfile(fout,dtype=dt_block, count=2)
fout.close()

# Store data in DataFrame
for k in range(2):                  # order = (1,2) x integType = (1,2)
    order = out_data[k]['info']['order']
    Itype = out_data[k]['info']['integType']
    if Itype == 1:        
        typex = 'inexact'
    else:
        typex = 'exact'
    
    for d in out_data[k]['data']:
        N = d[0]
        a = [d[i] for i in range(1,len(d))]
        df.loc[idx['Diffusion','CG',typex,order,N],cols] = a


# Format the data when displaying the DataFrame

fstr = {'cfl' : "{:.2f}".format, 
        '2-norm' : '{:.4e}'.format,}
        
df_out = df.loc[idx['Diffusion','CG',['inexact'],:],:]
df_out.style.format(fstr)